## Feature Extraction Notebook
---

This notebook is used to extract features from the data. The audio is divided into windows of 1 second. The features extracted from each audio window are:
- MFCC (Mel-Frequency Cepstral Coefficients): 20 coefficients
- Chroma STFT: 1 coefficient
- RMS (Root Mean Square) Energy: 1 coefficient
- Spectral Centroid: 1 coefficient
- Spectral Bandwidth: 1 coefficient
- Spectral Roll-off: 1 coefficient
- Zero Crossing Rate: 1 coefficient

In [ ]:
# import all the functions
from utils import *
import numpy as np

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

# -------- tqdm DARK THEME --------

### Features Extraction

In [ ]:
# set the paths to data
BASE_DIR = '../dataset/'
ARTIFACTS_DIR = BASE_DIR + 'artifacts/'
EXTRAHLS_DIR = BASE_DIR + 'extrahls/'
MURMURS_DIR = BASE_DIR + 'murmurs/'
NORMALS_DIR = BASE_DIR + 'normals/'
EXTRASTOLES_DIR = BASE_DIR + 'extrastoles/'

# paths to save the features
FEATURES_DIR = '../features/'

In [ ]:
# FEATURES EXTRACTION
paths = [ARTIFACTS_DIR, EXTRAHLS_DIR, MURMURS_DIR, NORMALS_DIR, EXTRASTOLES_DIR]
window_length = 1
sample_rate = 44100
n_mfcc = 20
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for i, path in tqdm(enumerate(paths), desc=f'Extracting features...'):
     
     if os.path.exists(FEATURES_DIR + str(window_length) + 's' + '.npz'):
          print('The features have already been extracted')
     else:
          features = extract_features(path, label = i, frame_length = window_length, sample_rate=sample_rate, n_mfcc=n_mfcc)
          
          # Stack the features into a single tensor
          features = torch.cat(features, dim=0)
          print(f'The shape of the {path} features tensor is: {features.shape}')
          
          # Save the features to a file
          names = ['artifacts', 'extrahls', 'murmurs', 'normals', 'extrastoles']
          name = f'{names[i]}_{window_length}.npz'
          save_features(features, FEATURES_DIR, name)
    
     print('Features extracted and saved')

          